<font size=6>Développer une preuve de concept</font>  
<font size=5>Classification d'image avec le papier Learning What and Where to Transfer  
Partie 1 : Améliorer les performances de mon CNN et de VGG16 grâce à EfficientNetB4</font>



---

**Vérification de l'environnement**

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
# PATH_DRIVE : to change according to your Google Drive folders
PATH_DRIVE = "/content/drive/My Drive/MachineLearning/ML07"
# IMAGES_DRIVE : to get access to previously loaded images
IMAGES_DRIVE = "/content/drive/My Drive/MachineLearning/ML06/Images"

In [ ]:
if IN_COLAB:
    print("Le notebook est exécuté sur Google Colab")
else:
    print("Le notebook est exécuté en local")

Le notebook est exécuté sur Google Colab


In [ ]:
if IN_COLAB:
    from google.colab import drive, files
    drive.mount("/content/drive")

Mounted at /content/drive


---
## <font color=blue>Notebook set-up</font>

**Importation des librairies**

In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import datetime
import random as python_random

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
import csv

In [ ]:
if IN_COLAB:
    sys.path.append(PATH_DRIVE)
    os.chdir(PATH_DRIVE)
    import sf_graphiques as sfg
    import sf_small_cnn as sf_target
    import sf_efficientnet_b4 as sf_source
    import sf_vgg16 as sf_vgg
    import sf_pytorch_loaders as sf_load
    import sf_meta_optimizers as sf_optim
    import sf_l2t_ww as sf_l2t
else:
    import modules_perso.sf_graphiques as sfg
    import modules_perso.sf_small_cnn as sf_target
    import modules_perso.sf_efficientnet_b4 as sf_source
    import modules_perso.sf_vgg16 as sf_vgg
    import modules_perso.sf_pytorch_loaders as sf_load
    import modules_perso.sf_meta_optimizers as sf_optim
    import modules_perso.sf_l2t_ww as sf_l2t

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**Notebook set-up**

In [ ]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = fig_id + "." + fig_extension
    if IN_COLAB:
        path = PATH_DRIVE + "/" + path
    #print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, dpi=resolution)

In [ ]:
RANDOM_SEED = 42
BATCH_SIZE = 32

In [ ]:
def reset_random_seeds():
    np.random.seed(RANDOM_SEED)
    python_random.seed(RANDOM_SEED)
    torch.manual_seed(40)

In [ ]:
reset_random_seeds()

In [ ]:
if torch.cuda.device_count() >= 1:
    print(torch.cuda.get_device_name(0))
else:
    print("No Cuda Device")
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

Tesla T4


**Personal Methods**

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def accuracy_topk(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(1.0 / batch_size))
    return res

In [ ]:
def accuracy_top1(outputs_data, labels):
    _, pred = torch.max(outputs_data, 1)
    correct = (pred == labels).sum().item()
    return correct / labels.size(0)

In [ ]:
def get_validation(model, loader):
    acc = AverageMeter()
    loss = AverageMeter()
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            y_pred = model.forward(x)
            loss.update(F.cross_entropy(y_pred, y))
            acc.update(accuracy_top1(y_pred.data, y))
    return loss.avg, acc.avg

In [ ]:
def transferlearning_on_targettask(mymodel, mycriterion, myoptimizer, myloaders,
                                   mysave_name, nb_epochs=50, patience=0):
    csv_filename = os.path.join(PATH_DRIVE, "logs/{}.csv".format(mysave_name))
    csv_memory = open(csv_filename, "w")  # Write the headers to the file
    writer = csv.writer(csv_memory)
    writer.writerow(["epoch", "validation_loss", "training_loss",
                      "validation_accuracy", "training_accuracy"])
    csv_memory.close()

    best_acc = 0.0
    lag_best = 0
    for epoch in range(nb_epochs):
        mymodel.train()
        epoch_acc = AverageMeter()
        epoch_loss = AverageMeter()
        for x, y in myloaders[0]:
            x, y = x.to(device), y.to(device)
            myoptimizer.zero_grad()
            y_pred = mymodel.forward(x)
            loss = mycriterion(y_pred, y)
            loss.backward()
            myoptimizer.step()
            epoch_loss.update(loss.item())
            epoch_acc.update(accuracy_top1(y_pred.data, y))
        training_loss = epoch_loss.avg
        training_accuracy = epoch_acc.avg
        
        validation_loss, validation_accuracy = get_validation(mymodel, myloaders[2])
        if validation_accuracy > best_acc:
            best_acc = validation_accuracy
            lag_best = 0
            torch.save(mymodel.state_dict(), "models/{}.pth".format(mysave_name))
        else:
            lag_best += 1

        print("[Epoch : {}]  [Training accuracy = {:.2%}] [Validation accuracy = {:.2%}] [Best = {:.2%}]  [Training loss = {:.3f}] [Validation loss = {:.3f}]"\
              .format(epoch, training_accuracy, validation_accuracy, best_acc, training_loss, validation_loss))
        csv_memory = open(csv_filename, "a")  # Write the results to the file
        writer = csv.writer(csv_memory)
        writer.writerow([epoch, validation_loss, training_loss,
                        validation_accuracy, training_accuracy])
        csv_memory.close()

        if (patience > 0) & (lag_best > patience):
            print("Learning ended by early-stop")
            break
        # next epoch

---
## 1\. Performance du modèle Baseline

Je calcule la performance du modèle CNN très simple que j'ai créé lors du traitement initial de ce problème.  
J'ajoute une couche de Batch Normalization après chaque couche de convolution pour résoudre un problème d'explosion du gradient rencontré lors de mon premier essai de la méthode l2t-ww avec ce modèle comme cible.

In [ ]:
model = sf_target.mycnn3(num_classes=120)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), eps=1e-7)
# note : eps=1e-7 to get the same epsilon than Tensorflow

In [ ]:
loaders = sf_load.get_dataset(IMAGES_DRIVE, model, mini_data=0.1, stratify=True,
                              batch_size=128, with_data_augmentation=False)

In [ ]:
transferlearning_on_targettask(model, criterion, optimizer, loaders,
                               mysave_name="baseline_model",
                               nb_epochs=1000, patience=50)

[Epoch : 0]  [Training accuracy = 0.64%] [Validation accuracy = 1.54%] [Best = 1.54%]  [Training loss = 4.951] [Validation loss = 4.796]
[Epoch : 1]  [Training accuracy = 1.19%] [Validation accuracy = 0.93%] [Best = 1.54%]  [Training loss = 4.786] [Validation loss = 4.786]
[Epoch : 2]  [Training accuracy = 1.19%] [Validation accuracy = 1.05%] [Best = 1.54%]  [Training loss = 4.788] [Validation loss = 4.778]
[Epoch : 3]  [Training accuracy = 0.21%] [Validation accuracy = 1.11%] [Best = 1.54%]  [Training loss = 4.772] [Validation loss = 4.770]
[Epoch : 4]  [Training accuracy = 1.83%] [Validation accuracy = 1.53%] [Best = 1.54%]  [Training loss = 4.759] [Validation loss = 4.764]
[Epoch : 5]  [Training accuracy = 2.07%] [Validation accuracy = 1.80%] [Best = 1.80%]  [Training loss = 4.737] [Validation loss = 4.758]
[Epoch : 6]  [Training accuracy = 1.57%] [Validation accuracy = 1.47%] [Best = 1.80%]  [Training loss = 4.727] [Validation loss = 4.754]
[Epoch : 7]  [Training accuracy = 2.33%] 

J'ai sauvegardé les meilleurs paramètres, car ils pourront constituer un bon point de départ pour les tests suivants.

---
## 2\. Transfert d'EfficientNetB4 vers le modèle Baseline

### 2.1. Premier essai : le modèle cible n'est pas pré-entraîné

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_target.mycnn3(num_classes=120)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)

In [ ]:
l2t_model.train(IMAGES_DRIVE, epochs=50, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_mycnn_mini")

[2020-11-13 17:35:26,834] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-aae3090b-91c1-4814-b659-ecfe8712cb56.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-13 18:32:10,542] [main] Best model is saved
[2020-11-13 18:32:10,612] [main] Last model is saved
[2020-11-13 18:32:10,613] [main] [Epoch : 1]  [Training accuracy = 0.76%] [Validation accuracy = 0.76%] [Best = 0.76%]  [Training loss = 5.334] [Validation loss = 5.354]
[2020-11-13 18:39:57,163] [main] Last model is saved
[2020-11-13 18:39:57,167] [main] [Epoch : 2]  [Training accuracy = 0.86%] [Validation accuracy = 0.76%] [Best = 0.76%]  [Training loss = 5.347] [Validation loss = 5.371]
[2020-11-13 18:47:44,071] [main] Best model is saved
[2020-11-13 18:47:44,141] [main] Last model is saved
[2020-11-13 18:47:44,144] [main] [Epoch : 3]  [Training accuracy = 0.76%] [Validation accuracy = 1.12%] [Best = 1.12%]  [Training loss = 4.936] [Validation loss = 4.946]
[2020-11-13 18:55:32,014] [main] Last model is saved
[2020-11-13 18:55:32,020] [main] [Epoch : 4]  [Training accuracy = 0.91%] [Validation accuracy = 0.91%] [Best = 1.12%]  [Training loss = 4.855] [Validation loss = 4.859]
[2

La performance est moins bonne avec la méthode l2t-ww depuis EfficientNetB4, qu'avec une optimisation de base avec Adam.

Je fais un autre test en chargeant le modèle cible pré-entraîné.

### 2.2. Deuxième essai : le modèle cible est pré-entraîné

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_target.mycnn3(num_classes=120, pretrained=True,
                                url_pretrained="models/baseline_cnn.pth")

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)

In [ ]:
l2t_model.train(IMAGES_DRIVE, epochs=200, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_mycnn_pretrained_mini")

[2020-11-13 22:37:39,390] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-aae3090b-91c1-4814-b659-ecfe8712cb56.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-13 22:45:23,572] [main] Best model is saved
[2020-11-13 22:45:23,651] [main] Last model is saved
[2020-11-13 22:45:23,652] [main] [Epoch : 1]  [Training accuracy = 0.91%] [Validation accuracy = 0.79%] [Best = 0.79%]  [Training loss = 5.398] [Validation loss = 5.433]
[2020-11-13 22:53:07,311] [main] Best model is saved
[2020-11-13 22:53:07,382] [main] Last model is saved
[2020-11-13 22:53:07,385] [main] [Epoch : 2]  [Training accuracy = 1.14%] [Validation accuracy = 1.12%] [Best = 1.12%]  [Training loss = 4.970] [Validation loss = 4.980]
[2020-11-13 23:00:50,840] [main] Last model is saved
[2020-11-13 23:00:50,844] [main] [Epoch : 3]  [Training accuracy = 0.91%] [Validation accuracy = 0.79%] [Best = 1.12%]  [Training loss = 4.905] [Validation loss = 4.914]
[2020-11-13 23:08:34,608] [main] Last model is saved
[2020-11-13 23:08:34,612] [main] [Epoch : 4]  [Training accuracy = 0.91%] [Validation accuracy = 0.94%] [Best = 1.12%]  [Training loss = 4.805] [Validation loss = 4.816]
[2

La performance ne s'améliore pas en basant l'entraînement sur le modèle pré-entraîné.  
Le modèle cible est trop simple pour apprendre correctement du transfert de connaissance.

---
## 3\. Transfert d'EfficientNetB4 vers VGG16

J'essaie désormais de voir l'efficacité de la méthode l2t-ww avec comme réseau cible VGG16 et comme réseau source EfficientNetB4.

### 3.1. Premier essai : VGG16 n'est pas pré-entraîné

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16(num_classes=120, original_classifier=False,
                            pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)

In [ ]:
l2t_model.train(IMAGES_DRIVE, epochs=200, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vgg_mini")

[2020-11-14 09:02:50,788] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-be98f530-0030-4bbd-ba03-9efb33eacb40.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-14 09:58:07,520] [main] Best model is saved
[2020-11-14 09:58:09,175] [main] Last model is saved
[2020-11-14 09:58:09,178] [main] [Epoch : 1]  [Training accuracy = 0.76%] [Validation accuracy = 0.73%] [Best = 0.73%]  [Training loss = nan] [Validation loss = nan]
[2020-11-14 10:20:44,457] [main] Last model is saved
[2020-11-14 10:20:44,459] [main] [Epoch : 2]  [Training accuracy = 0.76%] [Validation accuracy = 0.73%] [Best = 0.73%]  [Training loss = nan] [Validation loss = nan]


Je constate une explosion du gradient dès la première époque. J'exécute un nouvel essai avec un réseau cible pré-entraîné afin de voir si cela permet de limiter l'instabilité du gradient.

### 3.2. Deuxième essai : VGG16 est pré-entraîné

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16(num_classes=120, original_classifier=False,
                            pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)

In [ ]:
l2t_model.train(IMAGES_DRIVE, epochs=200, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vgg_pretrained_mini")

[2020-11-14 11:12:05,532] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-549bc2b4-c269-4194-a2d6-7ea68dc978db.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-14 12:05:39,138] [main] Best model is saved
[2020-11-14 12:05:39,812] [main] Last model is saved
[2020-11-14 12:05:39,814] [main] [Epoch : 1]  [Training accuracy = 0.76%] [Validation accuracy = 0.73%] [Best = 0.73%]  [Training loss = nan] [Validation loss = nan]
[2020-11-14 12:18:09,408] [main] Last model is saved
[2020-11-14 12:18:09,412] [main] [Epoch : 2]  [Training accuracy = 0.76%] [Validation accuracy = 0.73%] [Best = 0.73%]  [Training loss = nan] [Validation loss = nan]
[2020-11-14 12:30:38,652] [main] Last model is saved
[2020-11-14 12:30:38,655] [main] [Epoch : 3]  [Training accuracy = 0.76%] [Validation accuracy = 0.73%] [Best = 0.73%]  [Training loss = nan] [Validation loss = nan]
[2020-11-14 12:43:07,598] [main] Last model is saved
[2020-11-14 12:43:07,602] [main] [Epoch : 4]  [Training accuracy = 0.76%] [Validation accuracy = 0.73%] [Best = 0.73%]  [Training loss = nan] [Validation loss = nan]


Je constate une nouvelle fois une explosion du gradient. Je dois utiliser VGG16 avec Batch Normalization si je veux pouvoir tester la méthode l2t-ww.

---
## 4\. Transfert d'EfficientNetB4 vers VGG16 avec Batch Normalization

### 4.1. Pré-entraînement de VGG16 avec Batch Normalization

J'utilise VGG16 avec Batch Normalization avec une sortie en Global Average Pooling et une classification avec une couche dense composée de 120 sorties.  
J'optimise ce modèle sur les données du Stanford Dogs Dataset en comparant la performance des algorithmes Adam et SGD-Nesterov.

In [ ]:
model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                       pretrained=True)
model.freeze_layers(last_layer=-1)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), eps=1e-7)
# note : eps=1e-7 to get the same epsilon than Tensorflow

0 : features
freezed
1 : avgpool
freezed
2 : classifier
unfreezed


In [ ]:
loaders = sf_load.get_dataset(IMAGES_DRIVE, model, mini_data=0.1, stratify=True,
                              batch_size=64, with_data_augmentation=False)

In [ ]:
transferlearning_on_targettask(model, criterion, optimizer, loaders, "vgg16bn_120_adam")

[Epoch : 0]  [Training accuracy = 1.79%] [Validation accuracy = 8.03%] [Best = 8.03%]  [Training loss = 4.742] [Validation loss = 4.604]
[Epoch : 1]  [Training accuracy = 22.39%] [Validation accuracy = 25.42%] [Best = 25.42%]  [Training loss = 4.484] [Validation loss = 4.431]
[Epoch : 2]  [Training accuracy = 43.71%] [Validation accuracy = 34.46%] [Best = 34.46%]  [Training loss = 4.276] [Validation loss = 4.266]
[Epoch : 3]  [Training accuracy = 57.16%] [Validation accuracy = 41.46%] [Best = 41.46%]  [Training loss = 4.078] [Validation loss = 4.107]
[Epoch : 4]  [Training accuracy = 67.18%] [Validation accuracy = 45.68%] [Best = 45.68%]  [Training loss = 3.887] [Validation loss = 3.955]
[Epoch : 5]  [Training accuracy = 74.77%] [Validation accuracy = 49.03%] [Best = 49.03%]  [Training loss = 3.704] [Validation loss = 3.812]
[Epoch : 6]  [Training accuracy = 78.54%] [Validation accuracy = 51.66%] [Best = 51.66%]  [Training loss = 3.529] [Validation loss = 3.676]
[Epoch : 7]  [Training 

In [ ]:
model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                       pretrained=True)
model.freeze_layers(last_layer=-1)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)

0 : features
freezed
1 : avgpool
freezed
2 : classifier
unfreezed


In [ ]:
transferlearning_on_targettask(model, criterion, optimizer, loaders,
                               "vgg16bn_120_sgd", nb_epochs=200, patience=20)

[Epoch : 0]  [Training accuracy = 1.91%] [Validation accuracy = 5.61%] [Best = 5.61%]  [Training loss = 4.781] [Validation loss = 4.755]
[Epoch : 1]  [Training accuracy = 9.66%] [Validation accuracy = 13.49%] [Best = 13.49%]  [Training loss = 4.729] [Validation loss = 4.707]
[Epoch : 2]  [Training accuracy = 17.82%] [Validation accuracy = 17.90%] [Best = 17.90%]  [Training loss = 4.673] [Validation loss = 4.659]
[Epoch : 3]  [Training accuracy = 23.18%] [Validation accuracy = 20.34%] [Best = 20.34%]  [Training loss = 4.617] [Validation loss = 4.612]
[Epoch : 4]  [Training accuracy = 27.98%] [Validation accuracy = 22.63%] [Best = 22.63%]  [Training loss = 4.562] [Validation loss = 4.566]
[Epoch : 5]  [Training accuracy = 30.80%] [Validation accuracy = 24.89%] [Best = 24.89%]  [Training loss = 4.509] [Validation loss = 4.521]
[Epoch : 6]  [Training accuracy = 34.15%] [Validation accuracy = 26.58%] [Best = 26.58%]  [Training loss = 4.456] [Validation loss = 4.476]
[Epoch : 7]  [Training a

La meilleure performance est obtenue avec le SGD-Nesterov, qui semble de plus avoir encore un potentiel d'amélioration. Cette base est suffisante pour le pré-entraînement du modèle cible, qui sera de toute façon impactée par le transfert de connaissance.

### 4.2. Premier essai : VGG16bn n'est pas pré-entraîné

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)

In [ ]:
l2t_model.train(IMAGES_DRIVE, epochs=200, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_mini")

[2020-11-16 13:26:44,443] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-41a777f9-369d-49e1-879a-db699b2f6ff7.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-16 14:13:08,001] [main] Best model is saved
[2020-11-16 14:13:08,693] [main] Last model is saved
[2020-11-16 14:13:08,695] [main] [Epoch : 1]  [Training accuracy = 1.89%] [Validation accuracy = 1.24%] [Best = 1.24%]  [Training loss = 4.949] [Validation loss = 5.065]
[2020-11-16 14:41:06,249] [main] Last model is saved
[2020-11-16 14:41:06,252] [main] [Epoch : 2]  [Training accuracy = 1.59%] [Validation accuracy = 0.82%] [Best = 1.24%]  [Training loss = 5.361] [Validation loss = 5.621]
[2020-11-16 15:08:59,899] [main] Last model is saved
[2020-11-16 15:08:59,902] [main] [Epoch : 3]  [Training accuracy = 1.52%] [Validation accuracy = 1.03%] [Best = 1.24%]  [Training loss = 5.605] [Validation loss = 6.068]
[2020-11-16 15:36:51,556] [main] Last model is saved
[2020-11-16 15:36:51,559] [main] [Epoch : 4]  [Training accuracy = 2.98%] [Validation accuracy = 1.24%] [Best = 1.24%]  [Training loss = 5.494] [Validation loss = 6.114]
[2020-11-16 16:04:35,022] [main] Best model is saved
[2

L'apprentissage démarre assez lentement. Je regarde si le démarrage est plus rapide est prometteur avec le modèle pré-entraîné, car chaque époque est très longue, même en utilisant seulement 10% des données d'entraînement.

### 4.3. Deuxième essai : VGG16bn est pré-entraîné

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)
target_model.load_state_dict(torch.load("models/vgg16bn_120_sgd.pth"))

<All keys matched successfully>

In [ ]:
loaders = sf_load.get_dataset(IMAGES_DRIVE, target_model, mini_data=0.1, stratify=True,
                              batch_size=64, with_data_augmentation=False)
model = target_model.to(device)
get_validation(model, loaders[2])

(tensor(1.6716, device='cuda:0'), 0.6692994505494506)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)

In [ ]:
l2t_model.train(IMAGES_DRIVE, epochs=200, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_trained_mini")

[2020-11-16 21:13:17,437] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-f77f78c4-7212-41a4-a1c9-eebb4f6b3447.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-16 21:47:46,159] [main] Best model is saved
[2020-11-16 21:47:46,812] [main] Last model is saved
[2020-11-16 21:47:46,817] [main] [Epoch : 1]  [Training accuracy = 2.50%] [Validation accuracy = 1.64%] [Best = 1.64%]  [Training loss = 5.775] [Validation loss = 6.389]
[2020-11-16 22:13:04,974] [main] Best model is saved
[2020-11-16 22:13:05,659] [main] Last model is saved
[2020-11-16 22:13:05,663] [main] [Epoch : 2]  [Training accuracy = 6.29%] [Validation accuracy = 2.28%] [Best = 2.28%]  [Training loss = 4.984] [Validation loss = 6.011]
[2020-11-16 22:38:22,646] [main] Best model is saved
[2020-11-16 22:38:23,279] [main] Last model is saved
[2020-11-16 22:38:23,286] [main] [Epoch : 3]  [Training accuracy = 9.82%] [Validation accuracy = 3.40%] [Best = 3.40%]  [Training loss = 4.571] [Validation loss = 6.129]
[2020-11-16 23:03:38,437] [main] Last model is saved
[2020-11-16 23:03:38,440] [main] [Epoch : 4]  [Training accuracy = 13.66%] [Validation accuracy = 3.06%] [Best = 3.40%]

Je continue avec cette base qui commence son apprentissage plus rapidement.  
Je dois relancer plusieurs fois l'apprentissage, dû aux limitations en temps de Google Colab.

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)
l2t_model.load_parameters("ckpt_last_effb4_to_vggbn_trained_mini.pth")

Model loaded : best accuracy = 8.68% after 22 epochs


In [ ]:
l2t_model.train(IMAGES_DRIVE, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_trained_mini")

[2020-11-17 10:35:58,805] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-323f1809-fe0c-4e23-a1e2-8a216c12815a.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-17 11:34:46,169] [main] Last model is saved
[2020-11-17 11:34:46,174] [main] [Epoch : 23]  [Training accuracy = 71.89%] [Validation accuracy = 8.34%] [Best = 8.68%]  [Training loss = 1.326] [Validation loss = 10.964]
[2020-11-17 12:01:36,875] [main] Last model is saved
[2020-11-17 12:01:36,878] [main] [Epoch : 24]  [Training accuracy = 63.74%] [Validation accuracy = 7.28%] [Best = 8.68%]  [Training loss = 2.017] [Validation loss = 12.095]
[2020-11-17 12:28:26,841] [main] Last model is saved
[2020-11-17 12:28:26,844] [main] [Epoch : 25]  [Training accuracy = 64.55%] [Validation accuracy = 7.92%] [Best = 8.68%]  [Training loss = 1.900] [Validation loss = 12.620]
[2020-11-17 12:55:15,591] [main] Best model is saved
[2020-11-17 12:55:16,299] [main] Last model is saved
[2020-11-17 12:55:16,301] [main] [Epoch : 26]  [Training accuracy = 70.86%] [Validation accuracy = 8.71%] [Best = 8.71%]  [Training loss = 1.534] [Validation loss = 11.907]
[2020-11-17 13:22:05,304] [main] Last model

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)
l2t_model.load_parameters("ckpt_last_effb4_to_vggbn_trained_mini.pth")

Model loaded : best accuracy = 11.62% after 43 epochs


In [ ]:
l2t_model.train(IMAGES_DRIVE, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_trained_mini")

[2020-11-17 23:25:21,675] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-9fef629b-5e81-445e-8799-e77b74816e66.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-18 00:22:42,030] [main] Last model is saved
[2020-11-18 00:22:42,033] [main] [Epoch : 44]  [Training accuracy = 90.23%] [Validation accuracy = 10.41%] [Best = 11.62%]  [Training loss = 0.396] [Validation loss = 10.876]
[2020-11-18 00:49:11,440] [main] Last model is saved
[2020-11-18 00:49:11,442] [main] [Epoch : 45]  [Training accuracy = 91.06%] [Validation accuracy = 10.92%] [Best = 11.62%]  [Training loss = 0.396] [Validation loss = 10.470]
[2020-11-18 01:15:38,903] [main] Last model is saved
[2020-11-18 01:15:38,907] [main] [Epoch : 46]  [Training accuracy = 88.86%] [Validation accuracy = 10.29%] [Best = 11.62%]  [Training loss = 0.465] [Validation loss = 10.904]
[2020-11-18 01:42:05,687] [main] Last model is saved
[2020-11-18 01:42:05,692] [main] [Epoch : 47]  [Training accuracy = 92.95%] [Validation accuracy = 11.07%] [Best = 11.62%]  [Training loss = 0.292] [Validation loss = 10.025]
[2020-11-18 02:08:32,407] [main] Last model is saved
[2020-11-18 02:08:32,410] [main] [E

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"

In [ ]:
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)
l2t_model.load_parameters("ckpt_last_effb4_to_vggbn_trained_mini.pth")

Model loaded : best accuracy = 12.26% after 64 epochs


In [ ]:
l2t_model.train(IMAGES_DRIVE, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_trained_mini")

[2020-11-18 09:31:04,114] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-842d6815-ac90-4974-8d0d-69d946fadbb5.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-18 10:18:38,200] [main] Last model is saved
[2020-11-18 10:18:38,202] [main] [Epoch : 65]  [Training accuracy = 96.21%] [Validation accuracy = 11.17%] [Best = 12.26%]  [Training loss = 0.125] [Validation loss = 7.588]
[2020-11-18 10:46:56,736] [main] Last model is saved
[2020-11-18 10:46:56,738] [main] [Epoch : 66]  [Training accuracy = 97.12%] [Validation accuracy = 11.17%] [Best = 12.26%]  [Training loss = 0.088] [Validation loss = 7.390]
[2020-11-18 11:15:32,714] [main] Last model is saved
[2020-11-18 11:15:32,717] [main] [Epoch : 67]  [Training accuracy = 97.05%] [Validation accuracy = 11.04%] [Best = 12.26%]  [Training loss = 0.088] [Validation loss = 7.330]
[2020-11-18 11:44:09,656] [main] Last model is saved
[2020-11-18 11:44:09,659] [main] [Epoch : 68]  [Training accuracy = 96.89%] [Validation accuracy = 11.35%] [Best = 12.26%]  [Training loss = 0.095] [Validation loss = 7.236]
[2020-11-18 12:12:46,372] [main] Last model is saved
[2020-11-18 12:12:46,374] [main] [Epoch

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)
l2t_model.load_parameters("ckpt_last_effb4_to_vggbn_trained_mini.pth")

Model loaded : best accuracy = 13.56% after 84 epochs


In [ ]:
l2t_model.train(IMAGES_DRIVE, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_trained_mini")

[2020-11-18 21:21:50,708] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-7a8c89c0-11a6-424c-aa0a-6d6d0c3cfe8a.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-18 22:21:09,394] [main] Last model is saved
[2020-11-18 22:21:09,398] [main] [Epoch : 85]  [Training accuracy = 99.62%] [Validation accuracy = 13.50%] [Best = 13.56%]  [Training loss = 0.026] [Validation loss = 5.434]
[2020-11-18 22:48:33,795] [main] Best model is saved
[2020-11-18 22:48:34,423] [main] Last model is saved
[2020-11-18 22:48:34,427] [main] [Epoch : 86]  [Training accuracy = 99.70%] [Validation accuracy = 14.05%] [Best = 14.05%]  [Training loss = 0.023] [Validation loss = 5.360]
[2020-11-18 23:15:55,217] [main] Last model is saved
[2020-11-18 23:15:55,219] [main] [Epoch : 87]  [Training accuracy = 99.62%] [Validation accuracy = 13.83%] [Best = 14.05%]  [Training loss = 0.024] [Validation loss = 5.310]
[2020-11-18 23:43:14,938] [main] Last model is saved
[2020-11-18 23:43:14,941] [main] [Epoch : 88]  [Training accuracy = 99.62%] [Validation accuracy = 13.77%] [Best = 14.05%]  [Training loss = 0.025] [Validation loss = 5.263]
[2020-11-19 00:10:35,384] [main] Last m

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)
l2t_model.load_parameters("ckpt_last_effb4_to_vggbn_trained_mini.pth")

Model loaded : best accuracy = 14.71% after 104 epochs


In [ ]:
l2t_model.train(IMAGES_DRIVE, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_trained_mini")

[2020-11-19 08:23:42,791] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-9960d625-40bd-4b45-b248-3a6af69735b6.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-19 09:22:18,631] [main] Best model is saved
[2020-11-19 09:22:19,303] [main] Last model is saved
[2020-11-19 09:22:19,307] [main] [Epoch : 105]  [Training accuracy = 99.70%] [Validation accuracy = 14.78%] [Best = 14.78%]  [Training loss = 0.024] [Validation loss = 4.770]
[2020-11-19 09:48:36,432] [main] Best model is saved
[2020-11-19 09:48:37,125] [main] Last model is saved
[2020-11-19 09:48:37,127] [main] [Epoch : 106]  [Training accuracy = 99.70%] [Validation accuracy = 14.81%] [Best = 14.81%]  [Training loss = 0.024] [Validation loss = 4.752]
[2020-11-19 10:14:54,784] [main] Last model is saved
[2020-11-19 10:14:54,787] [main] [Epoch : 107]  [Training accuracy = 99.70%] [Validation accuracy = 14.68%] [Best = 14.81%]  [Training loss = 0.024] [Validation loss = 4.739]
[2020-11-19 10:41:11,801] [main] Last model is saved
[2020-11-19 10:41:11,803] [main] [Epoch : 108]  [Training accuracy = 99.70%] [Validation accuracy = 14.81%] [Best = 14.81%]  [Training loss = 0.023] [Validat

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)
l2t_model.load_parameters("ckpt_last_effb4_to_vggbn_trained_mini.pth")

Model loaded : best accuracy = 15.23% after 125 epochs


In [ ]:
l2t_model.train(IMAGES_DRIVE, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_trained_mini")

[2020-11-19 20:32:24,155] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-7b59f465-a852-42bf-ae62-764237908366.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-19 21:27:16,148] [main] Last model is saved
[2020-11-19 21:27:16,151] [main] [Epoch : 126]  [Training accuracy = 99.77%] [Validation accuracy = 15.11%] [Best = 15.23%]  [Training loss = 0.022] [Validation loss = 4.496]
[2020-11-19 21:53:09,878] [main] Last model is saved
[2020-11-19 21:53:09,880] [main] [Epoch : 127]  [Training accuracy = 99.77%] [Validation accuracy = 15.11%] [Best = 15.23%]  [Training loss = 0.022] [Validation loss = 4.487]
[2020-11-19 22:19:01,898] [main] Last model is saved
[2020-11-19 22:19:01,901] [main] [Epoch : 128]  [Training accuracy = 99.77%] [Validation accuracy = 15.23%] [Best = 15.23%]  [Training loss = 0.022] [Validation loss = 4.477]
[2020-11-19 22:44:51,943] [main] Best model is saved
[2020-11-19 22:44:52,618] [main] Last model is saved
[2020-11-19 22:44:52,622] [main] [Epoch : 129]  [Training accuracy = 99.77%] [Validation accuracy = 15.35%] [Best = 15.35%]  [Training loss = 0.022] [Validation loss = 4.470]
[2020-11-19 23:10:42,939] [main] La

In [ ]:
model_efficientnetb4_url = PATH_DRIVE + "/models/efficientnet-b4-6ed6700e.pth"
source_model = sf_source.efficientnetb4(pretrained=True, max_num_features=512,
                                        folder_saves="models",
                                        url_pretrained=model_efficientnetb4_url)
target_model = sf_vgg.vgg16bn(num_classes=120, original_classifier=False,
                              pretrained=False)

In [ ]:
l2t_model = sf_l2t.WhatWhere(source_model, target_model, device=device)
l2t_model.load_parameters("ckpt_last_effb4_to_vggbn_trained_mini.pth")

Model loaded : best accuracy = 15.50% after 138 epochs


In [ ]:
l2t_model.train(IMAGES_DRIVE, batch_size=8,
                mini_data=0.1, with_data_augmentation=False,
                early_stop=True, save_name="effb4_to_vggbn_trained_mini")

[2020-11-20 08:17:11,747] [main] /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-51d3c411-fa9b-47fc-a8bb-580976512fb1.json


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


[2020-11-20 08:59:51,725] [main] Best model is saved
[2020-11-20 08:59:52,383] [main] Last model is saved
[2020-11-20 08:59:52,388] [main] [Epoch : 139]  [Training accuracy = 99.77%] [Validation accuracy = 15.53%] [Best = 15.53%]  [Training loss = 0.021] [Validation loss = 4.405]
[2020-11-20 09:25:48,478] [main] Last model is saved
[2020-11-20 09:25:48,481] [main] [Epoch : 140]  [Training accuracy = 99.77%] [Validation accuracy = 15.50%] [Best = 15.53%]  [Training loss = 0.021] [Validation loss = 4.400]
[2020-11-20 09:51:45,343] [main] Last model is saved
[2020-11-20 09:51:45,346] [main] [Epoch : 141]  [Training accuracy = 99.77%] [Validation accuracy = 15.50%] [Best = 15.53%]  [Training loss = 0.020] [Validation loss = 4.393]
[2020-11-20 10:17:39,829] [main] Best model is saved
[2020-11-20 10:17:40,507] [main] Last model is saved
[2020-11-20 10:17:40,510] [main] [Epoch : 142]  [Training accuracy = 99.77%] [Validation accuracy = 15.59%] [Best = 15.59%]  [Training loss = 0.021] [Validat

Après 5 jours d'entraînement, je mets un terme à cette optimisation :
- elle est trop gourmande en ressources par rapport à mes moyens ;
- elle est trop gourmande en temps par rapport à mes objectifs ;
- elle ne semble pas disposer d'un potentiel de performance suffisant (training loss très faible alors que la validation accuracy est encore très insuffisante).